<table align="left"><td>
  <a target="_blank"  href="https://github.com/Dhruv0208/Research-Project/Preparing_Dataset.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Dhruv0208/Research-Project/Preparing_Dataset.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

## Importing important Libraries

In [1]:
%matplotlib inline
import cv2
from PIL import Image
from os import path
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import pandas as pd
import os
import csv

## Function for converting Nested list to flat list

In [2]:
catnms = []
def flatlist(l):
    for x in l:
        if type(x) == list:
            flatlist(x)
        else:
            catnms.append(x) 

## DATA IN FORMAT
Base_Path<br>
&emsp;&emsp;|-train2017<br>
&emsp;&emsp;|-val2017<br>
&emsp;&emsp;|-annotations_trainval2017<br>
&emsp;&emsp;&emsp;&emsp;|-instances_train2017<br>
&emsp;&emsp;&emsp;&emsp;|-instances_val2017<br>

Some of the categories are not present in the default annotations file. Use the Categories.txt file to get all the categories.


Converting categories names into list


In [3]:
l = []
with open('Categories', newline='') as inputfile:
    for row in csv.reader(inputfile):
        l.append(row)
flatlist(l)

CSV files created from the following codes will have column names, you have exclude them to use them for training.
That can be done either by editing the csv after it created by deleting the row or from the code by removing columns argument

## Training csv file

In [4]:
# setting up directory and file location

dataDir='./annotations_trainval2017'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
# column names
cols=["image_path","left","top","right","bottom","cat_ID","cat_Name"]
# creating class for annotation file
coco = COCO(annFile)
# display COCO supercategories
catNms = catnms
cats = coco.loadCats(coco.getCatIds())
nms = set([cat['supercategory'] for cat in cats])
# print('COCO supercategories: \n{}'.format(','.join("'" + i + "'" for i in nms)))
# creating the list of bounding boxes, category IDs and category names

for i in catNms:
    catIds = coco.getCatIds(catNms=['i'])
    imgIds = coco.getImgIds(catIds=catIds)
    images = coco.loadImgs(imgIds)
lst = []
for im in images:
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    for i in range(len(anns)):
        lst.append([im['file_name'],  
                     int(round(anns[i]['bbox'][0])), int(round(anns[i]['bbox'][1])),
                     int(round(anns[i]['bbox'][0] + anns[i]['bbox'][2])),  
                     int(round(anns[i]['bbox'][1] + anns[i]['bbox'][3])),anns[i]['category_id'],
                     catnms[anns[i]['category_id']-1]])

df = pd.DataFrame(lst, columns = cols)
df.to_csv('annotations_train.csv', index = False)

loading annotations into memory...
Done (t=11.05s)
creating index...
index created!


## Validation csv file

In [5]:
# setting up directory and file location
dataDir_val='./annotations_trainval2017'
dataType_val='val2017'
annFile_val='{}/instances_{}.json'.format(dataDir,dataType)
# creating class for annotation file
coco = COCO(annFile_val)
# display COCO supercategories
catNms = catnms
cats = coco.loadCats(coco.getCatIds())
nms = set([cat['supercategory'] for cat in cats])
# print('COCO supercategories: \n{}'.format(','.join("'" + i + "'" for i in nms)))
# creating the list of bounding boxes, category IDs and category names

for i in catNms:
    catIds = coco.getCatIds(catNms=['i'])
    imgIds = coco.getImgIds(catIds=catIds)
    images = coco.loadImgs(imgIds)
lst = []
for im in images:
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    for i in range(len(anns)):
        lst.append([im['file_name'],  
                     int(round(anns[i]['bbox'][0])), int(round(anns[i]['bbox'][1])),
                     int(round(anns[i]['bbox'][0] + anns[i]['bbox'][2])),  
                     int(round(anns[i]['bbox'][1] + anns[i]['bbox'][3])),anns[i]['category_id'],
                     catnms[anns[i]['category_id']-1]])

df = pd.DataFrame(lst, columns = cols)
df.to_csv('annotations_val.csv', index = False)